# **Crawling Data**

Crawling data adalah sebuah proses pengambilan data dari suatu sumber internet. Dalam hal ini saya mencrawling data dari twitter menggunakan twint. Data yang saya peroleh adalah sebanyak 80 data. Data tersebut saya saring sehingga menjadi 79 data karena terdapat beberapa data yang tidak penting. Data tersebut merupakan tweet dari tanggal 12-10-2022.



In [1]:
%%capture
!git clone --depth=1 https://github.com/twintproject/twint.git
%cd twint
!pip3 install . -r requirements.txt

In [2]:
%%capture
!pip install nest-asyncio #install library nest-asyncio

In [3]:
%%capture
!pip install aiohttp==3.7.0 #install aiohttp

## Twint

Twint adalah alat scraping data Twitter yang canggih ditulis dengan Python yang memungkinkan untuk scrapping Tweet dari profil Twitter tanpa menggunakan API Twitter. Twint menggunakan operator pencarian Twitter untuk scrapping Tweet dari pengguna tertentu, scrapping Tweet yang berkaitan dengan topik tertentu, tagar & tren, atau memilah informasi sensitif dari Tweet seperti email dan nomor telepon.

In [4]:
import nest_asyncio # import nest_asyncio
nest_asyncio.apply() #digunakan sekali untuk mengaktifkan tindakan serentak dalam notebook jupyter.
import twint #untuk import twint

## Hasil Crawling Data Twitter

In [12]:
c = twint.Config() #membuat variable c
c.Search = 'tragedi kanjuruhan' #key word untuk data
c.Pandas = True
c.Limit = 80 #mencrawl 80 data
twint.run.Search(c) #run

1579994741818593281 2022-10-12 00:37:55 +0000 <kaltenglima> Polri Beberkan Penyebab Korban Tragedi Kanjuruhan Meninggal   https://t.co/BeVo2U0gvU
1579994667944333312 2022-10-12 00:37:38 +0000 <penjagadapur> tuan jokowi, mengapa sampeyan tidak meminta maaf dan menystakan bertanggung jawab kepada ratusan korban mati dan atau luka luka dalam Tragedi Kanjuruhan?#dimanarasakemanusiaansampeyan?!
1579994545395093505 2022-10-12 00:37:09 +0000 <ZunanArfian> Kapolres Malang AKBP Putu Kholis Aryana duduk bersimpuh di depan pintu masuk pemeriksaan tiket, memanjatkan doa bagi korban Tragedi Kanjuruhan, di Stadion Kanjuruhan, Kabupaten Malang, Selasa (11/10/2022) siang. Hal ini dilakukan di sela-sela kunjungannya ke Stadion Kanjuruhan  https://t.co/WAa03qGnNN
1579994530660896768 2022-10-12 00:37:05 +0000 <Athaya_Abida> Kalangan industri komersialisasi olahraga nasional memuji hasil lobi @erickthohir ke FIFA yang membuat federasi sepak bola internasional itu urung memberikan sanksi berat kepada Indon

In [13]:
Tweets_df = twint.storage.panda.Tweets_df
Tweets_df["tweet"].to_csv("kanjuruhann.csv") #menyimpan ke kanjuruhan.csv

# **Preposesing**

Preprocessing adalah tahap untuk "membersihkan data". data - data tersebut dibersihkan sehingga menjadi data yang bagus untuk diolah.

In [7]:
%%capture
!pip install numpy #untuk install numpy

In [8]:
%%capture
!pip install pandas #untuk install pandas

In [9]:
%%capture
!pip install nltk #untuk install nltk

In [10]:
%%capture
!pip install scikit-learn #untuk install scikit-learn

## **Import Library yang digunakan**



In [11]:
import numpy as np #import numpy
#Library untuk mengelola data dalam Dataframe
import pandas as pd

#Lbrary untuk Preprocessing
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords  #stopwords
from nltk import word_tokenize # tokenizing

#Untuk membuat vektor dan TFIDF
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer, CountVectorizer

#Untuk melakukan proses SVD
from sklearn.decomposition import TruncatedSVD

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [14]:
Tweets_df = pd.read_csv("https://raw.githubusercontent.com/maulidhan190081/file/main/kanjuruhannn.csv") #mengambil data dari link

Tweets_df #print data

,tweet
0,Polri Beberkan Penyebab Korban Tragedi Kanjuru...
1,"tuan jokowi, mengapa sampeyan tidak meminta ma..."
2,Kapolres Malang AKBP Putu Kholis Aryana duduk ...
3,Kalangan industri komersialisasi olahraga nasi...
4,Klopun nanti liga jalan lagi dgn kapasitas 50%...
...,...
74,Korban Meninggal dalam Tragedi Kanjuruhan Bert...
75,Korban Meninggal dalam Tragedi Kanjuruhan Bert...
76,Presiden RI Joko Widodo akan membentuk tim khu...
77,"Kepala dinas kesehatan Kab. Malang Wiyanto : ""..."


## **Mengecilkan Semua Huruf**

In [17]:
Tweets_df['tweet']=Tweets_df['tweet'].str.replace(',', '') #menghilangkan koma
Tweets_df['tweet']=Tweets_df['tweet'].str.lower() #mengecilkan huruf
Tweets_df['tweet'] #print data kolom tweet

0     polri beberkan penyebab korban tragedi kanjuru...
1     tuan jokowi mengapa sampeyan tidak meminta maa...
2     kapolres malang akbp putu kholis aryana duduk ...
3     kalangan industri komersialisasi olahraga nasi...
4     klopun nanti liga jalan lagi dgn kapasitas 50%...
                            ...                        
74    korban meninggal dalam tragedi kanjuruhan bert...
75    korban meninggal dalam tragedi kanjuruhan bert...
76    presiden ri joko widodo akan membentuk tim khu...
77    kepala dinas kesehatan kab. malang wiyanto : "...
78    tgipf ungkap pengakuan lib : laga malam hari a...
Name: tweet, Length: 79, dtype: object

In [18]:
#Untuk menghapus angka
import re

#Untuk menghilangkan Punctuation
import string

## **Menghilangkan karakter spesial**

In [19]:
def remove_PTA_special(text):
    # remove tab, new line, ans back slice
    text = text.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\',"")
    # remove non ASCII (emoticon, chinese word, .etc)
    text = text.encode('ascii', 'replace').decode('ascii')
    # remove mention, link, hashtag
    text = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", text).split())
    # remove incomplete URL
    return text.replace("http://", " ").replace("https://", " ")
                
Tweets_df['tweet'] =Tweets_df['tweet'].apply(remove_PTA_special)

## **Menghilangkan Angka**

In [20]:
#remove number
def remove_number(text):
    return  re.sub(r"\d+", "", text)

Tweets_df['tweet'] = Tweets_df['tweet'].apply(remove_number)

## **Menghilangkan Tanda Baca**

In [21]:
#remove punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans("","",string.punctuation))

Tweets_df['tweet'] = Tweets_df['tweet'].apply(remove_punctuation)

## **Menghilangkan Spasi Di Awal dan Akhir**

In [22]:
#remove whitespace leading & trailing
def remove_whitespace_LT(text):
    return text.strip()

Tweets_df['tweet'] = Tweets_df['tweet'].apply(remove_whitespace_LT)

## **Menjadikan 1 spasi**

In [23]:
#remove multiple whitespace into single whitespace
def remove_whitespace_multiple(text):
    return re.sub('\s+',' ',text)

Tweets_df['tweet'] = Tweets_df['tweet'].apply(remove_whitespace_multiple)

## **Menghapus Char**

In [24]:
# remove single char
def remove_singl_char(text):
    return re.sub(r"\b[a-zA-Z]\b", "", text)

Tweets_df['tweet'] = Tweets_df['tweet'].apply(remove_singl_char)

## **Word Tokenize**

In [25]:
# NLTK word tokenize 
def word_tokenize_wrapper(text):
    return word_tokenize(text)

Tweets_df['tweet'] = Tweets_df['tweet'].apply(word_tokenize_wrapper)
Tweets_df['tweet']

0     [polri, beberkan, penyebab, korban, tragedi, k...
1     [tuan, jokowi, mengapa, sampeyan, tidak, memin...
2     [kapolres, malang, akbp, putu, kholis, aryana,...
3     [kalangan, industri, komersialisasi, olahraga,...
4     [klopun, nanti, liga, jalan, lagi, dgn, kapasi...
                            ...                        
74    [korban, meninggal, dalam, tragedi, kanjuruhan...
75    [korban, meninggal, dalam, tragedi, kanjuruhan...
76    [presiden, ri, joko, widodo, akan, membentuk, ...
77    [kepala, dinas, kesehatan, kab, malang, wiyant...
78    [tgipf, ungkap, pengakuan, lib, laga, malam, h...
Name: tweet, Length: 79, dtype: object

## **Stopwords**

In [26]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [27]:
#Mendapatkan stopword indonesia
list_stopwords = stopwords.words('indonesian')

#Menghapus Stopword dari list token
def stopwords_removal(words):
    return [word for word in words if word not in list_stopwords]
Tweets_df['tweet'] = Tweets_df['tweet'].apply(stopwords_removal)

Tweets_df['tweet']

0     [polri, beberkan, penyebab, korban, tragedi, k...
1     [tuan, jokowi, sampeyan, maaf, menystakan, ber...
2     [kapolres, malang, akbp, putu, kholis, aryana,...
3     [kalangan, industri, komersialisasi, olahraga,...
4     [klopun, liga, jalan, dgn, kapasitas, ya, pend...
                            ...                        
74    [korban, meninggal, tragedi, kanjuruhan, berta...
75    [korban, meninggal, tragedi, kanjuruhan, berta...
76    [presiden, ri, joko, widodo, membentuk, tim, k...
77    [kepala, dinas, kesehatan, kab, malang, wiyant...
78    [tgipf, pengakuan, lib, laga, malam, arema, fc...
Name: tweet, Length: 79, dtype: object

In [28]:
Tweets_df.to_csv('TextPreprocessing.csv')

In [29]:
dataTextPre = pd.read_csv('TextPreprocessing.csv')
vectorizer = CountVectorizer(min_df=1)
bag = vectorizer.fit_transform(dataTextPre['tweet'])

# **TF**

In [30]:
print(vectorizer.vocabulary_)

{'polri': 334, 'beberkan': 31, 'penyebab': 313, 'korban': 200, 'tragedi': 407, 'kanjuruhan': 162, 'meninggal': 260, 'tuan': 410, 'jokowi': 152, 'sampeyan': 357, 'maaf': 220, 'menystakan': 267, 'bertanggung': 47, 'ratusan': 345, 'mati': 230, 'luka': 217, 'kapolres': 165, 'malang': 225, 'akbp': 7, 'putu': 343, 'kholis': 186, 'aryana': 22, 'duduk': 94, 'bersimpuh': 44, 'pintu': 327, 'masuk': 227, 'pemeriksaan': 300, 'tiket': 403, 'memanjatkan': 237, 'doa': 91, 'stadion': 382, 'kabupaten': 158, 'selasa': 364, 'siang': 374, 'selasela': 365, 'kunjungannya': 205, 'kalangan': 160, 'industri': 134, 'komersialisasi': 194, 'olahraga': 285, 'nasional': 272, 'memuji': 245, 'hasil': 119, 'lobi': 214, 'fifa': 102, 'federasi': 101, 'sepak': 370, 'bola': 55, 'internasional': 135, 'urung': 419, 'sanksi': 358, 'berat': 33, 'indonesia': 132, 'paska': 292, 'klopun': 191, 'liga': 213, 'jalan': 145, 'dgn': 66, 'kapasitas': 163, 'ya': 433, 'pendapat': 305, 'pribadi': 337, 'sih': 376, 'ga': 104, 'klo': 190, 's

In [31]:
matrik_vsm=bag.toarray()
#print(matrik_vsm)
matrik_vsm.shape

(79, 436)

In [32]:
matrik_vsm[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [33]:
a=vectorizer.get_feature_names()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [34]:
print(len(matrik_vsm[:,1]))
#dfb =pd.DataFrame(data=matrik_vsm,index=df,columns=[a])
dataTF =pd.DataFrame(data=matrik_vsm,index=list(range(1, len(matrik_vsm[:,1])+1, )),columns=[a])
dataTF

79


,aamiin,abang,ade,afinta,air,aja,ajak,akbp,akibat,akp,...,vs,warga,warganet,waspadai,widodo,wiyanto,ws,ya,yg,zurich
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
76,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
77,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
78,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


# **KMeans**

K-Means Clustering adalah suatu metode penganalisaan data atau metode Data Mining yang melakukan proses pemodelan unssupervised learning dan menggunakan metode yang mengelompokan data berbagai partisi.

K means clustering merupakan metode algoritma dasar, yang diterapkan sebagai berikut:

1. Menentukan jumlah cluster
2. Menentukan centroid awal
3. Menghitung jarak data dengan centroid
4. Menentukan anggota cluster
5. menghitung rata-rata centroid tiap cluster
6. Ulangi no 3-5 sampai data cluster tidak berubah

Rumus Euclidean Distance:

<img src="https://latex.codecogs.com/gif.latex?\left&space;[&space;\left&space;(&space;x,y&space;\right&space;),\left&space;(&space;a,b&space;\right&space;)&space;\right&space;]=\sqrt{\left&space;(&space;x-a&space;\right&space;)^{2}&plus;\left&space;(&space;y-b&space;\right&space;)^{2}}" title="\left [ \left ( x,y \right ),\left ( a,b \right ) \right ]=\sqrt{\left ( x-a \right )^{2}+\left ( y-b \right )^{2}}" /> 


In [35]:
from sklearn.cluster import KMeans

kmeans =KMeans(n_clusters=2)
kmeans=kmeans.fit(dataTF)
prediksi=kmeans.predict(dataTF)
centroids = kmeans.cluster_centers_
centroids

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  FutureWarning,


array([[ 4.05405405e-02,  1.35135135e-02,  1.11022302e-16,
         2.70270270e-02,  8.10810811e-02,  4.05405405e-02,
         1.35135135e-02,  1.35135135e-02,  2.70270270e-02,
         2.70270270e-02,  1.35135135e-02,  1.35135135e-02,
         2.70270270e-02,  1.35135135e-02,  1.35135135e-02,
         5.40540541e-02,  1.35135135e-02,  1.35135135e-02,
         5.40540541e-02,  1.35135135e-02,  2.77555756e-17,
         1.35135135e-02,  1.35135135e-02,  1.35135135e-02,
         1.35135135e-02,  1.35135135e-02,  5.40540541e-02,
         1.35135135e-02,  1.35135135e-02,  1.35135135e-02,
         1.35135135e-02,  1.35135135e-02,  1.35135135e-02,
         2.70270270e-02,  1.35135135e-02,  1.35135135e-02,
         1.35135135e-02,  1.35135135e-02,  1.35135135e-02,
         6.75675676e-02,  1.35135135e-02,  1.35135135e-02,
         1.35135135e-02,  1.35135135e-02,  1.35135135e-02,
         1.35135135e-02,  8.10810811e-02,  4.05405405e-02,
         2.70270270e-02,  1.35135135e-02,  1.35135135e-0

In [36]:
df= pd.DataFrame(prediksi)
df

,0
0,0
1,0
2,0
3,0
4,0
...,...
74,0
75,0
76,0
77,0


In [37]:
centroid = pd.DataFrame(centroids)
centroid

,0,1,2,3,4,5,6,7,8,9,...,426,427,428,429,430,431,432,433,434,435
0,4.054054e-02,1.351351e-02,1.110223e-16,2.702703e-02,8.108108e-02,4.054054e-02,1.351351e-02,1.351351e-02,2.702703e-02,2.702703e-02,...,4.054054e-02,1.351351e-02,1.351351e-02,1.351351e-02,1.351351e-02,1.351351e-02,1.351351e-02,0.067568,1.621622e-01,1.351351e-02
1,-6.938894e-18,-1.734723e-18,1.600000e+00,-3.469447e-18,-1.387779e-17,-6.938894e-18,-1.734723e-18,-1.734723e-18,-3.469447e-18,-3.469447e-18,...,-6.938894e-18,-1.734723e-18,-1.734723e-18,-1.734723e-18,-1.734723e-18,-1.734723e-18,-1.734723e-18,0.000000,-2.775558e-17,-1.734723e-18


# **Crawling Berita**

In [ ]:
%%capture
!pip install PyPDF2

In [ ]:
%%capture
!pip install docx2txt

## Mengimport numpy, PyPDF2, docx2txt, sys

In [ ]:
import numpy as np
import PyPDF2
import docx2txt
import sys

In [ ]:
pdfFileObj = open('Jakarta.pdf', 'rb')
pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
pageObj = pdfReader.getPage(0)
document = pageObj.extractText()
document

'Jakarta, CNBC Indonesia - Seorang pejabat tinggi Rusia memperingatkan bahaya "eskalasi \nyang tidak terkendali" jika Barat terus mendukung Ukraina. Ini menjadi ancaman terbaru \nyang dikeluarkan Rusia ke sekutu internat ional Kyiv.  "Rusia akan dipaksa untuk mengambil \ntindakan pencegahan yang memadai, termasuk yang bersifat asimetris," kata Wakil Menteri \nLuar Negeri Sergei Ryabkov mengatakan kepada kantor berita milik negara Rusia RIA \nNovosti, dikutip CNBC International, S elasa (11/10/2022).  Ia menyinggung bagaimana \nMoskow menyesali bantuan skala besar yang sedang berlangsung ke Kyiv dari Barat . Ia pun \nmenyebut bentrokan langsung dengan AS dan NATO, meski tak dijelaskan lebih lanjut.  \n"Bentrokan langsung dengan AS dan NATO  bukan untuk kepentingan Rusia," tegasnya.  \n"Bahwa Washington dan ibu kota Barat lainnya sadar akan bahaya eskalasi yang tidak \nterkendali," tambahnya lagi.  Sementara itu, dalam pernyataan berbeda, Rusia juga \nmengatakan mengharapkan lebih banyak

## Parsing dokumen menjadi kalimat

In [ ]:
from nltk.tokenize.punkt import PunktSentenceTokenizer
def tokenize(document):
    # Kita memecahnya menggunakan  PunktSentenceTokenizer
    # 
    doc_tokenizer = PunktSentenceTokenizer()
    
    # metode tokenize() memanggil dokument kita
    # sebagai input dan menghasilkan daftar kalimat dalam dokumen
    
    # sentences_list adalah daftar masing masing kalimat dari dokumen yang ada.
    sentences_list = doc_tokenizer.tokenize(document)
    return sentences_list

In [ ]:
sentences_list = tokenize(document)

In [ ]:
for i in sentences_list:
    print('------')
    print(i)

------
Jakarta, CNBC Indonesia - Seorang pejabat tinggi Rusia memperingatkan bahaya "eskalasi 
yang tidak terkendali" jika Barat terus mendukung Ukraina.
------
Ini menjadi ancaman terbaru 
yang dikeluarkan Rusia ke sekutu internat ional Kyiv.
------
"Rusia akan dipaksa untuk mengambil 
tindakan pencegahan yang memadai, termasuk yang bersifat asimetris," kata Wakil Menteri 
Luar Negeri Sergei Ryabkov mengatakan kepada kantor berita milik negara Rusia RIA 
Novosti, dikutip CNBC International, S elasa (11/10/2022).
------
Ia menyinggung bagaimana 
Moskow menyesali bantuan skala besar yang sedang berlangsung ke Kyiv dari Barat .
------
Ia pun 
menyebut bentrokan langsung dengan AS dan NATO, meski tak dijelaskan lebih lanjut.
------
"Bentrokan langsung dengan AS dan NATO  bukan untuk kepentingan Rusia," tegasnya.
------
"Bahwa Washington dan ibu kota Barat lainnya sadar akan bahaya eskalasi yang tidak 
terkendali," tambahnya lagi.
------
Sementara itu, dalam pernyataan berbeda, Rusia juga 

In [ ]:
print ("Banyaknya kalimat = ", (len(sentences_list)))
sentences_list

Banyaknya kalimat =  13


['Jakarta, CNBC Indonesia - Seorang pejabat tinggi Rusia memperingatkan bahaya "eskalasi \nyang tidak terkendali" jika Barat terus mendukung Ukraina.',
 'Ini menjadi ancaman terbaru \nyang dikeluarkan Rusia ke sekutu internat ional Kyiv.',
 '"Rusia akan dipaksa untuk mengambil \ntindakan pencegahan yang memadai, termasuk yang bersifat asimetris," kata Wakil Menteri \nLuar Negeri Sergei Ryabkov mengatakan kepada kantor berita milik negara Rusia RIA \nNovosti, dikutip CNBC International, S elasa (11/10/2022).',
 'Ia menyinggung bagaimana \nMoskow menyesali bantuan skala besar yang sedang berlangsung ke Kyiv dari Barat .',
 'Ia pun \nmenyebut bentrokan langsung dengan AS dan NATO, meski tak dijelaskan lebih lanjut.',
 '"Bentrokan langsung dengan AS dan NATO  bukan untuk kepentingan Rusia," tegasnya.',
 '"Bahwa Washington dan ibu kota Barat lainnya sadar akan bahaya eskalasi yang tidak \nterkendali," tambahnya lagi.',
 'Sementara itu, dalam pernyataan berbeda, Rusia juga \nmengatakan mengh

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer

## TF

In [ ]:
cv = CountVectorizer()
cv_matrix = cv.fit_transform(sentences_list)
print(cv_matrix)

  (0, 43)	1
  (0, 20)	1
  (0, 37)	1
  (0, 92)	1
  (0, 80)	1
  (0, 105)	1
  (0, 86)	1
  (0, 62)	1
  (0, 8)	1
  (0, 33)	1
  (0, 110)	1
  (0, 103)	1
  (0, 100)	1
  (0, 44)	1
  (0, 12)	1
  (0, 102)	1
  (0, 63)	1
  (0, 106)	1
  (1, 86)	1
  (1, 110)	1
  (1, 38)	1
  (1, 67)	1
  (1, 4)	1
  (1, 99)	1
  (1, 29)	1
  :	:
  (11, 25)	1
  (11, 58)	1
  (11, 91)	1
  (11, 42)	1
  (11, 21)	1
  (11, 82)	1
  (11, 14)	1
  (11, 45)	1
  (11, 66)	1
  (11, 11)	1
  (11, 51)	1
  (12, 86)	1
  (12, 106)	1
  (12, 38)	1
  (12, 99)	1
  (12, 107)	1
  (12, 82)	1
  (12, 27)	1
  (12, 68)	1
  (12, 83)	1
  (12, 24)	1
  (12, 34)	1
  (12, 61)	1
  (12, 93)	1
  (12, 26)	1


In [ ]:
print ("Banyaknya kosa kata = ", len((cv.get_feature_names_out())))

Banyaknya kosa kata =  111


In [ ]:
print ("kosa kata = ", (cv.get_feature_names_out()))

kosa kata =  ['10' '11' '2022' 'akan' 'ancaman' 'as' 'asimetris' 'bagaimana' 'bahaya'
 'bahwa' 'bantuan' 'banyak' 'barat' 'bentrokan' 'berbeda' 'berita'
 'berlangsung' 'bersifat' 'besar' 'bukan' 'cnbc' 'dalam' 'dan' 'dari'
 'darurat' 'dengan' 'di' 'dibuat' 'dijelaskan' 'dikeluarkan' 'dikutip'
 'dipaksa' 'elasa' 'eskalasi' 'g7' 'ia' 'ibu' 'indonesia' 'ini' 'internat'
 'international' 'ional' 'itu' 'jakarta' 'jika' 'juga' 'kantor' 'kata'
 'ke' 'kepada' 'kepentingan' 'konfrontasi' 'kota' 'kyiv' 'lagi' 'lainnya'
 'langsung' 'lanjut' 'lebih' 'luar' 'memadai' 'membahas' 'memperingatkan'
 'mendukung' 'mengambil' 'mengatakan' 'mengharapkan' 'menjadi' 'menjelang'
 'menteri' 'menyebut' 'menyesali' 'menyinggung' 'meski' 'milik' 'moskow'
 'nato' 'negara' 'negeri' 'novosti' 'pejabat' 'pencegahan' 'pernyataan'
 'pertemuan' 'pun' 'ria' 'rusia' 'ryabkov' 'sadar' 'sedang' 'sekutu'
 'sementara' 'seorang' 'serangan' 'sergei' 'skala' 'tak' 'tambahnya'
 'tegasnya' 'terbaru' 'terkendali' 'termasuk' 'terus' 

## Matriks Similaritas

In [ ]:
normal_matrix = TfidfTransformer().fit_transform(cv_matrix)
print(normal_matrix.toarray())
print(normal_matrix.T.toarray)

[[0.         0.         0.         ... 0.         0.         0.1562289 ]
 [0.         0.         0.         ... 0.         0.         0.18955945]
 [0.17554387 0.17554387 0.17554387 ... 0.17554387 0.         0.20178593]
 ...
 [0.         0.         0.         ... 0.         0.2737021  0.18241605]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]
<bound method _cs_matrix.toarray of <111x13 sparse matrix of type '<class 'numpy.float64'>'
	with 203 stored elements in Compressed Sparse Column format>>


In [ ]:
normal_matrix.shape

(13, 111)

## Graph

In [ ]:
import networkx as nx

In [ ]:
res_graph = normal_matrix * normal_matrix.T
print(res_graph)

  (0, 11)	0.04992018207567556
  (0, 9)	0.02292472501898624
  (0, 7)	0.04992018207567556
  (0, 5)	0.02292472501898624
  (0, 12)	0.07253668752363857
  (0, 8)	0.07253668752363857
  (0, 10)	0.25879955185704245
  (0, 6)	0.25879955185704245
  (0, 3)	0.05001633813182891
  (0, 2)	0.08665227988870172
  (0, 1)	0.04806640083095473
  (0, 0)	0.9999999999999999
  (1, 11)	0.023251763370229772
  (1, 9)	0.027815585783412172
  (1, 7)	0.023251763370229772
  (1, 5)	0.027815585783412172
  (1, 12)	0.15311169806955632
  (1, 8)	0.15311169806955632
  (1, 10)	0.03457868588094215
  (1, 6)	0.03457868588094215
  (1, 3)	0.1669672553475922
  (1, 2)	0.062082774740978244
  (1, 1)	0.9999999999999997
  (1, 0)	0.04806640083095473
  (2, 11)	0.05991420849474964
  :	:
  (10, 2)	0.06939007393133981
  (10, 1)	0.03457868588094215
  (10, 0)	0.25879955185704245
  (11, 10)	0.03591228519865729
  (11, 6)	0.03591228519865729
  (11, 3)	0.03151378218908757
  (11, 4)	0.10360999828104903
  (11, 12)	0.07867564186512492
  (11, 9)	0.084080

In [ ]:
nx_graph = nx.from_scipy_sparse_matrix(res_graph)

In [ ]:
print('Banyaknya sisi {}'.format(nx_graph.number_of_edges()))

Banyaknya sisi 78


## Page Rank

In [ ]:
ranks = nx.pagerank(nx_graph)

In [ ]:
ranked_sentences = sorted(((ranks[i],s) for i,s in enumerate(sentences_list)), reverse=True)

## Top 10 Kalimat

In [ ]:
# Extract top 10 sentences as the summary
for i in range(10):
  print(ranked_sentences[i][1])

"Bentrokan langsung dengan AS dan NATO bukan untuk kepentingan Rusia," tegasnya.
"Bentrokan langsung dengan AS dan NATO  bukan untuk kepentingan Rusia," tegasnya.
"Bahwa Washington dan ibu kota Barat lainnya sadar akan bahaya eskalasi yang tidak 
terkendali," tambahnya lagi.
"Bahwa Washington dan ibu kota Barat lainnya sadar akan  bahaya eskalasi yang tidak 
terkendali," tambahnya lagi.
Sementara itu, dalam pernyataan berbeda, Rusia juga 
mengatakan mengharapkan lebih banyak "konfrontasi" dengan Barat.
Sementara itu, dalam pernyataan berbeda, Rusia juga 
mengatakan mengharapkan lebih banyak "konfrontasi " dengan Barat.
Pernyataan ini dibuat 
menjelang pertemuan darurat G7 untuk membahas serangan Rusia terbaru di Ukraina.
Pernyataan ini dibuat 
menjelang pertemuan darurat G7 untuk membahas serangan Rusia  terbaru di Ukraina.
Ia pun 
menyebut bentrokan langsung dengan AS dan NATO, meski tak dijelaskan lebih lanjut.
Jakarta, CNBC Indonesia - Seorang pejabat tinggi Rusia memperingatkan bah